# Diving into VesselFinder

## Enhancing the code - Part2

Let's assume that we are interested in collecting the data coming from several web pages (several vessels...).
We don't want to use complicated graphical toolkit, nor databases, but still want to share our tool in a friendlly way with our colleagues....

We create a google spreadsheet or an ethercalc one, which are both shareable as a csv file.

We'll need pandas, a python library that acts like Excel on steroïds to read this file, and enclose our previous script in a loop.


### Installing some packages

We will use 1 more package to scrape the data :  Pandas

In [2]:
pip install pandas --user

Note: you may need to restart the kernel to use updated packages.


We import the libraries, including pandas.... plus some extra things (random, time, path...).

In [34]:
import csv
import time
from datetime import datetime
import os.path
from random import randint
from time import sleep

In [10]:
import requests
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

The url changes for the spreadsheet one....

In [18]:
url_spread = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR5OEMAcXUby3T1GmrteVb_sQTEUP0rF9QRXlsl6SEL-7y9Ih30kTERgAQe0V4RMdVHn-QSYhcNYlZb/pub?gid=0&single=true&output=csv'


We reuse the previous script as a function in Python. We will then apply this function to all our urls....
The only change is that we also scrape the name of the vessel to name our files based on this name....

In [20]:
def retrieve_vessel(url):
    scrape_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    headers = {'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:81.0) Gecko/20100101 Firefox/81.0'}
    reqs = requests.get(url, headers=headers)
    soup = BeautifulSoup(reqs.text, 'lxml')
    data = soup.find_all('td')
    coordinates = data[21].get_text()
    lat = coordinates.split('/')[0]
    lon = coordinates.split('/')[1]
    date_tag = data[25]
    date_tag = str(date_tag)
    date_tag_soup = BeautifulSoup(date_tag, features="lxml")
    date_tag = date_tag_soup.td['data-title']
    date_tag = date_tag.replace(',', '').strip(' UTC')
    date_tag = datetime.strptime(date_tag, '%b %d %Y %H:%M')
    date_collect = date_tag.strftime('%Y-%m-%d')
    time_collect = date_tag.strftime('%H:%M')
    head_spd = data[19].get_text()
    heading = head_spd.split(' / ')[0]
    speed = head_spd.split(' / ')[1]
    
    
    filename_html = os.path.join('%s_AIS.html' % vessel_name)
    filename_csv = os.path.join('%s_AIS.csv' % vessel_name)
    
    with open(filename_html, "w", encoding='utf-8') as file:
        file.write(str(soup))
    with open(filename_csv, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow([scrape_time, lat, lon, date_collect, time_collect, heading, speed])

###  Iteration
We can now iterate this function threw our csv file using a loop. For each vessel, we will retrieve the same data, automatically. First we read the csv file a put it in a dataframe. Reading a csv file is smoother with Pandas!

In [31]:
ship_list = pd.read_csv(url_spread)


In [32]:
print(ship_list)

           name                                                url
0  Motivation_D  https://www.vesselfinder.com/vessels/MOTIVATIO...
1        Gulsun  https://www.vesselfinder.com/vessels/MSC-GULSU...


Then we read each row in a loop and apply our retrieve_vessel function to the url.
*important* : BE NICE TO APIs! Always place a random pause between too requests, or you'll be banned!

In [35]:
for index, row in ship_list.iterrows():
    url = (ship_list['url'][index])
    vessel_name = (ship_list['name'][index])
    retrieve_vessel(url)
    sleep(randint(1, 5))